# Flight Data Retrieval

In [1]:
import pandas as pd
import os
from natsort import os_sorted
from tqdm.notebook import tqdm
from sqlalchemy import create_engine, types
from sqlalchemy import text # to be able to pass string

## Raw Data Source: https://transtats.bts.gov/PREZIP/

## Preprocessing Steps

In [3]:
# select columns to keep
def cols_to_keep(flights_raw):
    columns_to_keep = [
        "FlightDate",
        "DepTime",
        "CRSDepTime",
        "DepDelay",
        "ArrTime",
        "CRSArrTime",
        "ArrDelay",
        "Reporting_Airline",
        "Tail_Number",
        "Flight_Number_Reporting_Airline",
        "Origin",
        "Dest",
        "AirTime",
        "ActualElapsedTime",
        "Distance",
        "Cancelled",
        "Diverted",
    ]
    flights = flights_raw.loc[:, columns_to_keep]
    return flights

In [4]:
# rename columns
def rename_cols(flights):
    new_column_names = {
        'FlightDate': 'flight_date',
        'DepTime': 'dep_time',
        'CRSDepTime': 'sched_dep_time',
        'DepDelay': 'dep_delay',
        'ArrTime': 'arr_time',
        'CRSArrTime': 'sched_arr_time',
        'ArrDelay': 'arr_delay',
        'Reporting_Airline': 'airline',
        'Tail_Number': 'tail_number',
        'Flight_Number_Reporting_Airline': 'flight_number',
        'Origin': 'origin',
        'Dest': 'dest',
        'AirTime': 'air_time',
        'ActualElapsedTime': 'actual_elapsed_time',
        'Distance': 'distance',
        'Cancelled': 'cancelled',
        'Diverted': 'diverted'
    }
    flights.rename(columns=new_column_names, inplace=True)
    return flights

In [5]:
# change datatype
def change_dtypes(flights):
    types_change = {
        'flight_date': 'datetime64[ns]',
        'dep_time': 'float64',
        'sched_dep_time': 'int16',
        'dep_delay': 'float64',
        'arr_time': 'float64',
        'sched_arr_time': 'int16',
        'arr_delay': 'float64',
        'airline': 'O',
        'tail_number': 'O',
        'flight_number': 'int64',
        'origin': 'O',
        'dest': 'O',
        'air_time': 'float64',
        'actual_elapsed_time': 'float64',
        'distance': 'int16',
        'cancelled': 'int16',
        'diverted': 'int16'
    }
    flights = flights.astype(types_change)
    return flights

## Reading the Data 

In [ ]:
# getting file names for the for-loop (sorted with function 'os_sorted' from the 'natsort' package)
data_files = os_sorted(os.listdir('./data/extracted/'))

In [8]:
# getting the period range for the big file name
year_month = [x.split(')_')[1][:-4] for x in data_files]
f'flights_from_{year_month[0]}_until_{year_month[-1]}'

'flights_from_2014_1_until_2016_3'

In [9]:
flights_list = []

for file in tqdm(data_files):
    print(file)
    print('reading...', end=" ")
    flights_raw = pd.read_csv(f'./data/extracted/{file}', low_memory = False) # read as a dataframe
    
    flights_select = cols_to_keep(flights_raw) # select columns to keep
    print('filter colums...', end=" ")
    flights_rename = rename_cols(flights_select) # rename columns
    print('rename colums...', end=" ")
    flights_dtypes = change_dtypes(flights_rename) # change data types
    print('change dtypes...', end=" ")
    
    flights_list.append(flights_dtypes) # add to the list of dateframes
    print('appended to flight_list\n')
    
print(f'Done. The list has {len(flights_list)} elements')

  0%|          | 0/15 [00:00<?, ?it/s]

On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2014_1.csv
reading... filter colums... rename colums... change dtypes... appended to flight_list

On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2014_2.csv
reading... filter colums... rename colums... change dtypes... appended to flight_list

On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2014_3.csv
reading... filter colums... rename colums... change dtypes... appended to flight_list

On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2014_10.csv
reading... filter colums... rename colums... change dtypes... appended to flight_list

On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2014_11.csv
reading... filter colums... rename colums... change dtypes... appended to flight_list

On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2014_12.csv
reading... filter colums... rename colums... change dtypes... appended to flight_list

On_Time_Reporting_Carrier_On_Time_Performan

In [10]:
flights_all = pd.concat(flights_list)

In [11]:
flights_all

,flight_date,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,airline,tail_number,flight_number,origin,dest,air_time,actual_elapsed_time,distance,cancelled,diverted
0,2014-01-30,935.0,940,-5.0,1051.0,1055,-4.0,AA,N006AA,2377,DFW,ICT,56.0,76.0,328,0,0
1,2014-01-31,951.0,940,11.0,1115.0,1055,20.0,AA,N003AA,2377,DFW,ICT,54.0,84.0,328,0,0
2,2014-01-01,1144.0,1135,9.0,1302.0,1300,2.0,AA,N002AA,2377,ICT,DFW,57.0,78.0,328,0,0
3,2014-01-02,1134.0,1135,-1.0,1253.0,1300,-7.0,AA,N002AA,2377,ICT,DFW,53.0,79.0,328,0,0
4,2014-01-03,1129.0,1135,-6.0,1244.0,1300,-16.0,AA,N014AA,2377,ICT,DFW,52.0,75.0,328,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479117,2016-03-02,551.0,555,-4.0,652.0,712,-20.0,UA,N402UA,1760,RNO,SFO,46.0,61.0,192,0,0
479118,2016-03-02,1752.0,1759,-7.0,2058.0,2120,-22.0,UA,N417UA,1762,MSY,IAD,111.0,126.0,955,0,0
479119,2016-03-02,2136.0,2140,-4.0,2309.0,2308,1.0,UA,N77510,1765,IAH,LAS,172.0,213.0,1222,0,0
479120,2016-03-02,1806.0,1740,26.0,2045.0,2032,13.0,UA,N462UA,1765,PHL,IAH,195.0,219.0,1325,0,0


We want only the data for the following airports: 
- NY - John F. Kennedy Airport 74486
- Washington D.C - Dulles International Airport 72403
- Philadelphia - Philadelphia International Airport 72408

In [12]:
# Filtering out the flights where origin is one of the above airports
flights_filtered = flights_all[flights_all['origin'].isin(['JFK', 'IAD', 'PHL'])]

In [13]:
# Filtered fligths to csv conversion
#flights_filtered.to_csv(f'./data/flights_from_{year_month[0]}_until_{year_month[-1]}.csv', index=False)

## Postgres Configuration

In [14]:
from dotenv import dotenv_values

config = dotenv_values()
pg_user = config['POSTGRES_USER']  # align the key label with your .env file !
pg_host = config['POSTGRES_HOST']
pg_port = config['POSTGRES_PORT']
pg_db = config['POSTGRES_DB']
pg_schema = config['POSTGRES_SCHEMA']
pg_pass = config['POSTGRES_PASS']

In [15]:
# Now building the URL with the values from the .env file

url = f'postgresql://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}'

In [16]:
# Create an engine to manage connections to the database
engine = create_engine(url, echo=False)

# Insert the data into the database
flights_filtered.to_sql('flights', engine, schema='clear_skies', if_exists='replace', index=False)


668

### Granting Permissions 

- Ran this in DBeaver to give permissions to colleagues

```sql
GRANT ALL PRIVILEGES 
ON ALL TABLES 
IN SCHEMA clear_skies 
TO anaisabelcosio, michaelraphael, maltestolle;


- Verifying Permissions

```sql
SELECT grantee, table_catalog, table_schema, table_name, privilege_type
FROM information_schema.role_table_grants
WHERE table_schema = 'clear_skies' AND table_name = 'flights'